In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
stats = pd.read_csv("../data/players_stats.csv") # player stats
tl = pd.read_csv("../data/primera-division.csv") # Transfer la liga
tp = pd.read_csv("../data/premier-league.csv") # transfers premier
ts = pd.read_csv("../data/serie-a.csv") # transfers serie a
tb = pd.read_csv("../data/bundesliga.csv") # transfers bundesliga

Check if the have the same strcuture

In [3]:
print("Transfers la liga shape:" , tl.shape)
print("Transfers premier shape:" , tp.shape)
print("Transfers serie a shape:" , ts.shape)
print("Transfers bundeliga shape:" , tb.shape)

Transfers la liga shape: (17255, 12)
Transfers premier shape: (23498, 12)
Transfers serie a shape: (28211, 12)
Transfers bundeliga shape: (14032, 12)


In [4]:
print("La liga columns",tl.columns)
print("premier columns",tp.columns)
print("premier columns",ts.columns)
print("premier columns",tb.columns)

La liga columns Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season'],
      dtype='object')
premier columns Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season'],
      dtype='object')
premier columns Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season'],
      dtype='object')
premier columns Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season'],
      dtype='object')


In [5]:
transf = pd.concat([tl,tp,ts,tb]) # merge both leagues transfers

In [6]:
transf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82996 entries, 0 to 14031
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   club_name           82996 non-null  object 
 1   player_name         82996 non-null  object 
 2   age                 82969 non-null  float64
 3   position            82996 non-null  object 
 4   club_involved_name  82996 non-null  object 
 5   fee                 82963 non-null  object 
 6   transfer_movement   82996 non-null  object 
 7   transfer_period     82996 non-null  object 
 8   fee_cleaned         33612 non-null  float64
 9   league_name         82996 non-null  object 
 10  year                82996 non-null  int64  
 11  season              82996 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 8.2+ MB


In [7]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50434 entries, 0 to 50433
Data columns (total 35 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      50434 non-null  int64  
 1   name            50434 non-null  object 
 2   club            50434 non-null  object 
 3   position        50434 non-null  object 
 4   apps            50434 non-null  object 
 5   mins            50434 non-null  int64  
 6   goals           50434 non-null  object 
 7   assists         50434 non-null  object 
 8   yel             50434 non-null  object 
 9   red             50434 non-null  object 
 10  shots           50434 non-null  object 
 11  ps%             50434 non-null  object 
 12  aerials_won     50434 non-null  object 
 13  motm            50434 non-null  object 
 14  rating          50434 non-null  float64
 15  tackles         50394 non-null  object 
 16  interceptions   50394 non-null  object 
 17  fouls           50394 non-null 

In [8]:
transf.describe(include="all")

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
count,82996,82996,82969.000000,82996,82996,82963,82996,82996,33612.000000,82996,82996.000000,82996
unique,205,20001,NaN,17,3397,2907,2,3,NaN,4,NaN,31
top,Udinese Calcio,Luis García,NaN,Centre-Forward,Without Club,loan transfer,out,Summer,NaN,Serie A,NaN,2014/2015
freq,1541,43,NaN,15557,1429,18855,45503,61514,NaN,28211,NaN,3852
mean,NaN,NaN,24.402705,NaN,NaN,NaN,NaN,NaN,3.176336,NaN,2009.006338,NaN
std,NaN,NaN,4.472954,NaN,NaN,NaN,NaN,NaN,7.829511,NaN,8.280058,NaN
min,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1992.000000,NaN
25%,NaN,NaN,21.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2003.000000,NaN
50%,NaN,NaN,24.000000,NaN,NaN,NaN,NaN,NaN,0.400000,NaN,2010.000000,NaN
75%,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,2016.000000,NaN


We want to look up for keys in order to make a left join between stats and transfers in order to add a column that indicates if the player has abandoned the team at the end of the season.

We will check club names, player names, season and league

### Club names

Note: We just have stats in sesaons 2009/2010 and above

In [9]:
seasons = ['2009/2010','2010/2011','2011/2012','2012/2013','2013/2014','2014/2015','2015/2016','2016/2017','2017/2018','2018/2019','2019/2020','2020/2021','2021/2022','2022/2023']
transf = transf[transf['season'].isin(seasons)]

In [10]:
clubs_transf = transf['club_name'].unique()
clubs_stats = stats['club'].unique()

In [11]:
print("unique clubs transf",len(clubs_transf))
print("unique clubs stats",len(clubs_stats))

unique clubs transf 149
unique clubs stats 142


In [12]:
print(np.setdiff1d(clubs_stats,clubs_transf))
print(len(np.setdiff1d(clubs_stats,clubs_transf)))

['AC Carpi' 'Almeria' 'Arsenal' 'Atalanta' 'Atletico' 'Augsburg'
 'Barcelona' 'Bari' 'Bayern' 'Benevento' 'Birmingham' 'Blackburn'
 'Blackpool' 'Bochum' 'Bologna' 'Bolton' 'Borussia M.Gladbach'
 'Bournemouth' 'Brentford' 'Brescia' 'Brighton' 'Burnley' 'Cadiz'
 'Cagliari' 'Cardiff' 'Catania' 'Celta Vigo' 'Cesena' 'Chelsea' 'Chievo'
 'Cordoba' 'Crotone' 'Darmstadt' 'Deportivo' 'Deportivo Alaves' 'Eibar'
 'Elche' 'Empoli' 'Espanyol' 'Everton' 'FC Koln' 'Fiorentina'
 'Fortuna Duesseldorf' 'Freiburg' 'Frosinone' 'Fulham' 'Genoa' 'Getafe'
 'Girona' 'Granada' 'Greuther Fuerth' 'Hamburg' 'Hannover' 'Hercules'
 'Hertha Berlin' 'Hoffenheim' 'Huddersfield' 'Hull' 'Ingolstadt' 'Inter'
 'Juventus' 'Kaiserslautern' 'Las Palmas' 'Lazio' 'Lecce' 'Leeds'
 'Leganes' 'Leicester' 'Levante' 'Leverkusen' 'Liverpool' 'Livorno'
 'Mainz' 'Malaga' 'Mallorca' 'Man City' 'Man Utd' 'Middlesbrough' 'Napoli'
 'Newcastle' 'Norwich' 'Novara' 'Nuernberg' 'Osasuna' 'Paderborn'
 'Palermo' 'Pescara' 'Portsmouth' 'QPR' 'RB

In [13]:
clubs_transf

array(['Racing Santander', 'Getafe CF', 'UD Almería', 'Sporting Gijón',
       'Málaga CF', 'Villarreal CF', 'Valencia CF',
       'Deportivo de La Coruña', 'RCD Espanyol Barcelona', 'CD Tenerife',
       'Atlético de Madrid', 'Athletic Bilbao', 'Real Madrid',
       'Sevilla FC', 'Real Valladolid CF', 'CA Osasuna', 'RCD Mallorca',
       'Real Zaragoza', 'Xerez CD', 'FC Barcelona', 'Hércules CF',
       'Levante UD', 'Real Sociedad', 'Granada CF', 'Rayo Vallecano',
       'Real Betis Balompié', 'Celta de Vigo', 'Elche CF', 'SD Eibar',
       'Córdoba CF', 'UD Las Palmas', 'Deportivo Alavés', 'CD Leganés',
       'Girona FC', 'SD Huesca', 'Cádiz CF', 'Manchester United',
       'Liverpool FC', 'Chelsea FC', 'Arsenal FC', 'Everton FC',
       'Aston Villa', 'Fulham FC', 'Tottenham Hotspur', 'West Ham United',
       'Manchester City', 'Wigan Athletic', 'Stoke City',
       'Bolton Wanderers', 'Portsmouth FC', 'Blackburn Rovers',
       'Sunderland AFC', 'Hull City', 'Wolverhampton Wande

It is easy to see what is going on, the club names in the transf dataframe have the complete name and the ones in the stats don´t. We will shorten them and remove the special characters

In [14]:
transf['clubs_short'] = transf['club_name'].apply(lambda x: x.strip().replace('CF','').replace('UD','')
                                                             .replace('CD','').replace('RCD','').replace('CA','')
                                                              .replace('SD','').replace('FC','').replace('á','a')
                                                              .replace('é','e').replace('í','i').replace('ó','o')
                                                              .replace('ú','u').replace('City','').replace('AC','')
                                                              .replace('United','Utd').replace('Town','')
                                                              .replace('SS','').replace('BC','').replace('AS','')
                                                              .replace('Calcio','').replace('US','').replace('BSC','')
                                                              .replace('VfB','').replace('SV','').replace('FSV','')
                                                              .replace('ST','').replace('RB','').replace('SC','')
                                                              .replace('SpvGG','').replace('VfL','').replace('96','')
                                                              .replace('98','').replace('04','').strip()
                                                             ) 


In [15]:
clubs_transf = transf['clubs_short'].unique() 
clubs_transf

array(['Racing Santander', 'Getafe', 'Almeria', 'Sporting Gijon',
       'Malaga', 'Villarreal', 'Valencia', 'Deportivo de La Coruña',
       'R Espanyol Barcelona', 'Tenerife', 'Atletico de Madrid',
       'Athletic Bilbao', 'Real Madrid', 'Sevilla', 'Real Valladolid',
       'Osasuna', 'R Mallorca', 'Real Zaragoza', 'Xerez', 'Barcelona',
       'Hercules', 'Levante', 'Real Sociedad', 'Granada',
       'Rayo Vallecano', 'Real Betis Balompie', 'Celta de Vigo', 'Elche',
       'Eibar', 'Cordoba', 'Las Palmas', 'Deportivo Alaves', 'Leganes',
       'Girona', 'Huesca', 'Cadiz', 'Manchester Utd', 'Liverpool',
       'Chelsea', 'Arsenal', 'Everton', 'Aston Villa', 'Fulham',
       'Tottenham Hotspur', 'West Ham Utd', 'Manchester',
       'Wigan Athletic', 'Stoke', 'Bolton Wanderers', 'Portsmouth',
       'Blackburn Rovers', 'Sunderland A', 'Hull',
       'Wolverhampton Wanderers', 'Birmingham', 'Burnley', 'Blackpool',
       'West Bromwich Albion', 'Newcastle Utd', 'Queens Park Rangers',
  

In [16]:
print(np.setdiff1d(clubs_stats,clubs_transf))
print(len(np.setdiff1d(clubs_stats,clubs_transf)))

['AC Carpi' 'AC Milan' 'Atletico' 'Bayern' 'Blackburn' 'Bologna' 'Bolton'
 'Borussia M.Gladbach' 'Bournemouth' 'Brighton' 'Catania' 'Celta Vigo'
 'Chievo' 'Deportivo' 'Espanyol' 'FC Koln' 'Fiorentina'
 'Fortuna Duesseldorf' 'Genoa' 'Greuther Fuerth' 'Hamburg' 'Hertha Berlin'
 'Hoffenheim' 'Inter' 'Kaiserslautern' 'Leeds' 'Leverkusen' 'Mainz'
 'Mallorca' 'Man City' 'Man Utd' 'Napoli' 'Newcastle' 'Novara' 'Nuernberg'
 'Paderborn' 'Parma Calcio 1913' 'Pescara' 'QPR' 'RBL' 'Real Betis'
 'SD Huesca' 'Salernitana' 'Sampdoria' 'Sheff Utd' 'Sunderland'
 'Tottenham' 'Union Berlin' 'Verona' 'West Brom' 'West Ham' 'Wigan'
 'Wolves']
53


Still not good enough. We will create a dictionary for the remaining clubs

In [17]:
clubs = {
    "Atletico de Madrid":"Atletico",
    "Blackburn Rovers":"Blackburn",
    "Bolton Wanderers":"Bolton",
    "A Bournemouth":"Bournemouth",
    "Brighton & Hove Albion":"Brighton",
    "Celta de Vigo":"Celta Vigo",
    "Deportivo de La Coruña":"Deportivo",
    "R Espanyol Barcelona":"Espanyol",
    "Leeds Utd":"Leeds",
    "R Mallorca":"Mallorca",
    "Manchester":"Man City",
    "Manchester Utd":"Man Utd",
    "Newcastle Utd":"Newcastle",
    "Queens Park Rangers":"QPR",
    "Real Betis Balompie":"Real Betis",
    "Huesca":"SD Huesca",
    "Sheffield Utd":"Sheff Utd",
    "Sunderland A":"Sunderland",
    "Tottenham Hotspur":"Tottenham",
    "West Bromwich Albion":"West Brom",
    "West Ham Utd":"West Ham",
    "Wigan Athletic":"Wigan",
    "Wolverhampton Wanderers":"Wolves",
    "Milan":"AC Milan",
    "Carpi  1909":"AC Carpi",
    "Bologna  1909":"Bologna",
    "Chievo Verona":"Chievo",
    "A Fiorentina":"Fiorentina",
    "Catania S":"Catania",
    "Genoa C":"Genoa",
    "Inter Milan":"Inter",
    "Hellas Verona":"Verona",
    "C Napoli":"Napoli",
    "Novara  1908":"Novara",
    "Delfino Pescara 1936":"Pescara",
    "Salernitana 1919":"Salernitana",
    "UC Sampdoria":"Sampdoria",
    "Parma  1913":"Parma Calcio 1913",
    "Bayern Munich":"Bayern",
    "Borussia Mönchengladbach":"Borussia M.Gladbach",
    "Darmstadt 98":"Darmstadt",
    "1.  Köln":"FC Koln",
    "Fortuna Düsseldorf":"Fortuna Duesseldorf",
    "SpVgg Greuther Fürth":"Greuther Fuerth",
    "Hamburger":"Hamburg",
    "Hertha":"Hertha Berlin",
    "TSG 1899 Hoffenheim":"Hoffenheim",
    "1. Kaiserslautern":"Kaiserslautern",
    "Bayer  Leverkusen":"Leverkusen",
    "1.F Mainz 05":"Mainz",
    "1. Nuremberg":"Nuernberg",
    "Paderborn 07":"Paderborn",
    "Leipzig":"RBL",
    "1. Union Berlin":"Union Berlin"
}

In [18]:
transf = transf.replace({'clubs_short':clubs} )

In [19]:
clubs_transf = transf['clubs_short'].unique() 
print(np.setdiff1d(clubs_stats,clubs_transf))
print(len(np.setdiff1d(clubs_stats,clubs_transf)))

[]
0


### League

In [20]:
transf['league_name'].unique()

array(['Primera Division', 'Premier League', 'Serie A', '1 Bundesliga'],
      dtype=object)

In [21]:
stats['league'].unique()

array(['Bundesliga', 'LaLiga', 'Premier League', 'Serie A'], dtype=object)

Change league names in transf dataset

In [22]:
transf['league_name'] = transf['league_name'].replace('Primera Division','LaLiga')

### Player Names

In [23]:
transf.player_name.head(20)

7802           Álex Geijo
7803        Marc Torrejón
7804            Pape Diop
7805         Manuel Arana
7806          José Crespo
7807        Nasief Morris
7808          Luis García
7809             Henrique
7810                Xisco
7811       Sergio Canales
7812       Damián Ísmodes
7813      Juanjo Expósito
7814      Brian Sarmiento
7815               Vitolo
7816         Ebi Smolarek
7817          Iván Bolado
7818       Jonathan Valle
7819       Daniel Szetela
7820    Cristian Portilla
7821         Iván Marcano
Name: player_name, dtype: object

In [24]:
stats.name.head(20)

0               Arjen Robben
1            Stefan Kießling
2         Zvjezdan Misimovic
3                 Edin Dzeko
4            Claudio Pizarro
5             Paolo Guerrero
6                 Mesut Özil
7                      Naldo
8                 Toni Kroos
9                    Raffael
10    Bastian Schweinsteiger
11                Nuri Sahin
12             Kevin Kurányi
13              Burak Kaplan
14             Lucas Barrios
15                Zé Roberto
16             Thomas Müller
17               Marko Marin
18       Tranquillo Barnetta
19               Eljero Elia
Name: name, dtype: object

Looks like they can have the same nomenclature

In [25]:
transf[transf["player_name"] == "Javi Martínez"]

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,clubs_short
9510,Athletic Bilbao,Javi Martínez,23.0,Defensive Midfield,Bayern Munich,€40.00m,out,Summer,40.0,LaLiga,2012,2012/2013,Athletic Bilbao
16308,CA Osasuna,Javi Martínez,21.0,Attacking Midfield,CA Osasuna Prom,-,in,Summer,NaN,LaLiga,2021,2021/2022,Osasuna
16906,CA Osasuna,Javi Martínez,22.0,Attacking Midfield,Albacete,loan transfer,out,summer,NaN,LaLiga,2022,2022/2023,Osasuna
7881,Bayern Munich,Javi Martínez,23.0,Defensive Midfield,Athletic,€40.00m,in,Summer,40.0,1 Bundesliga,2012,2012/2013,Bayern
12901,Bayern Munich,Javi Martínez,32.0,Defensive Midfield,Qatar SC,free transfer,out,Summer,0.0,1 Bundesliga,2021,2021/2022,Bayern


## Season

We are just interested in the outs because we want to study the churn

In [26]:
transf['transfer_movement'].isnull().sum()

0

In [27]:
out = transf[transf['transfer_movement'] == 'out']

We have to substract one season to the signing season because the player has been signing with the last season stats. 

In [49]:
out['season_less1'] = out['season'].apply(lambda x: str(int(x.split('/')[0]) - 1) + '/' + str(int(x.split('/')[1]) - 1))

<ipython-input-49-edede6d37c22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['season_less1'] = out['season'].apply(lambda x: str(int(x.split('/')[0]) - 1) + '/' + str(int(x.split('/')[1]) - 1))


In [61]:
print("Out len is ", len(out))

Out len is  26416


## Join

Let´s see if the names have the same nomenclature. We will make an inner join and if the size of this inner is the same as the out dataframen then we will be lucky :) 

In [76]:
pd.set_option('display.max_columns', None)
pd.merge(out,stats,how='inner',left_on=['clubs_short','player_name','season_less1'],right_on=['club','name','season']).drop_duplicates()

,club_name,player_name,age,position_x,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season_x,clubs_short,season_less1,Unnamed: 0,name,club,position_y,apps,mins,goals,assists,yel,red,shots,ps%,aerials_won,motm,rating,tackles,interceptions,fouls,offsides_won,clearances,dribbled,blocks,own_goals,key_passes,dribblings,fouled,offsides,dispossed,bad_controls,avg_passes,crosses,long_passes,through_passes,league,season_y
0,RCD Espanyol Barcelona,Nicolás Pareja,26.0,Centre-Back,Spartak Moscow,€10.00m,out,Summer,10.0,LaLiga,2010,2010/2011,Espanyol,2009/2010,108,Nicolás Pareja,Espanyol,D(C),30,2639,-,-,9,3,0.5,73,1.6,-,7.03,2.5,2.7,1.4,0.7,6.6,0.4,0.6,-,0.2,0.2,0.7,-,0.3,-,43.5,0.1,4.4,0.1,LaLiga,2009/2010
2,RCD Espanyol Barcelona,Moisés Hurtado,29.0,Defensive Midfield,Olympiacos,€1.80m,out,Summer,1.8,LaLiga,2010,2010/2011,Espanyol,2009/2010,269,Moisés Hurtado,Espanyol,M(CR),30(1),2556,-,-,9,-,0.6,76.9,1.5,-,6.76,2.1,1.7,2.1,0.2,3.6,0.6,0.4,1,0.2,0.4,0.9,-,0.7,-,39.2,0.1,2.9,-,LaLiga,2009/2010
4,RCD Espanyol Barcelona,Fernando Marqués,25.0,Right Winger,Parma,Loan fee:€200Th.,out,Summer,0.2,LaLiga,2010,2010/2011,Espanyol,2009/2010,361,Fernando Marqués,Espanyol,M(CL),10(11),918,1,5,2,-,0.6,71.9,0.2,-,6.59,0.8,0.5,0.6,-,-,0.4,-,-,1,1,1.3,0.2,1,-,13.6,0.8,0.6,0.1,LaLiga,2009/2010
6,RCD Espanyol Barcelona,Raúl Tamudo,32.0,Centre-Forward,Real Sociedad,free transfer,out,Summer,0.0,LaLiga,2010,2010/2011,Espanyol,2009/2010,519,Raúl Tamudo,Espanyol,AM(L),4(2),378,-,-,1,-,1,79.4,-,-,6.09,0.3,0.7,1,-,-,0.7,-,-,0.2,-,0.5,1,0.7,-,10.5,-,0.2,-,LaLiga,2009/2010
8,RCD Espanyol Barcelona,Ben Sahar,20.0,Centre-Forward,Hapoel Tel Aviv,loan transfer,out,Summer,NaN,LaLiga,2010,2010/2011,Espanyol,2009/2010,478,Ben Sahar,Espanyol,FW,5(17),604,1,-,-,-,0.8,77.6,0.4,-,6.23,0.1,0.4,0.5,-,0.1,0.1,-,-,0.2,0.4,0.8,0.7,0.5,-,6.5,0.1,0.2,-,LaLiga,2009/2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15642,Hertha BSC,Niklas Stark,27.0,Centre-Back,Werder Bremen,free transfer,out,summer,0.0,1 Bundesliga,2022,2022/2023,Hertha Berlin,2021/2022,206,Niklas Stark,Hertha Berlin,DMC,22(4),2028,1,-,3,-,0.4,82.4,1.9,-,6.59,1.1,1.4,0.7,0.6,4.5,0.2,0.7,-,0.2,0.3,0.8,-,0.1,0.5,38.5,-,2.2,-,Bundesliga,2021/2022
15643,Hertha BSC,Lukas Klünter,26.0,Right-Back,Arm. Bielefeld,free transfer,out,summer,0.0,1 Bundesliga,2022,2022/2023,Hertha Berlin,2021/2022,408,Lukas Klünter,Hertha Berlin,M(R),2(3),193,-,-,1,-,-,74.3,0.6,-,6.23,0.2,0.4,1,-,0.4,0.8,-,-,0.4,0.6,0.4,-,0.4,0.8,14.8,0.4,1,-,Bundesliga,2021/2022
15644,Hertha BSC,Ishak Belfodil,30.0,Centre-Forward,Al-Gharafa SC,free transfer,out,summer,0.0,1 Bundesliga,2022,2022/2023,Hertha Berlin,2021/2022,224,Ishak Belfodil,Hertha Berlin,FW,17(9),1643,5,2,1,-,1.2,71.1,1.5,-,6.56,0.3,0.2,0.7,-,0.6,0.2,-,-,0.8,0.7,0.4,1,1.1,1.5,17.5,0.1,0.2,-,Bundesliga,2021/2022
15645,Hertha BSC,Luca Wollschläger,19.0,Centre-Forward,RW Essen,loan transfer,out,summer,NaN,1 Bundesliga,2022,2022/2023,Hertha Berlin,2021/2022,439,Luca Wollschläger,Hertha Berlin,Forward,0(2),14,-,-,-,-,0.5,83.3,-,-,6.16,0.5,-,0.5,-,-,0.5,-,-,0.5,-,-,-,0.5,-,3,-,-,-,Bundesliga,2021/2022
